# testing phase2b

In [10]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import numpy as np
from typing import Tuple, cast, Optional
import numpy as np
import random
import enum
from qiskit import Aer, QuantumRegister, ClassicalRegister, QuantumCircuit, execute

## auxiliary functions

In [31]:
class GuessStrategy(enum.Enum):
    one_bit_same_as_measured = 1
    two_bit_base = 2
    two_bit_neural_network = 3

In [32]:
def _prepare_initial_state_entangled(state_probability: float) -> Tuple[complex, complex, complex, complex]:
        """ Prepare initial state: computing 'y' as the amplitudes  """
        return (0, np.sqrt(state_probability), np.sqrt(1 - state_probability), 0)

def _guess_lambda_used_two_bit_strategy(counts: str) -> int:
    """ Decides which lambda was used on the real execution from the two 'counts' measured
        Setting eta0 >= eta1:
            * outcome 00 -> eta0 as the most probable (more attenuation)
            * outcome 01 -> eta1 as the most probable (less attenuation)
            * outcome 10 -> 50% chance, random choice
            * outcome 11 -> not possible, but in case we get it (from noisy simulation), 50% chance, random choice
    """
    if len(counts) != 2:
        raise ValueError('counts MUST be a two character length string')
    if counts == "00":
        return 0
    if counts == "01":
        return 1
    if counts == "10" or counts == "11":
        return random.choice([0, 1])
    raise ValueError("Accepted counts are '00', '01', '10', '11'")

def _convert_counts_to_eta_used(counts_dict: dict,
                                guess_strategy: GuessStrategy) -> int:
    """ Decides which eta was used on the real execution from the 'counts' measured
        based on the guess strategy that is required to use
    """
    if guess_strategy != GuessStrategy.two_bit_base:
        raise ValueError('Invalid Guess Strategy. Only GuessStrategy.two_bit_base supported')

    counts = get_measured_value_from_counts(counts_dict)
    return _guess_lambda_used_two_bit_strategy(counts)

def _compute_damping_channel(configuration: dict, eta_index: int) -> int:
    """ one-time execution of the two-qubit entangled amplitude damping circuit using the passed parameters
        Returns: the execution measured result: either 0 or 1
    """
    backend = Aer.get_backend('qasm_simulator')
    eta = configuration['eta_pair'][eta_index]
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = _prepare_initial_state_entangled(configuration['state_probability'])

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * eta, qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.rx(configuration['angle_rx'], qreg_q[1])
    circuit.ry(configuration['angle_ry'], qreg_q[1])
    circuit.barrier()
    circuit.measure([0, 1], creg_c)

    counts = execute(circuit, backend, shots=1).result().get_counts(circuit)
    return _convert_counts_to_eta_used(counts, guess_strategy=GuessStrategy.two_bit_base)

In [33]:
def reorder_pair(pair: Tuple[float, float]) -> Tuple[float, float]:
    if pair[0] < pair[1]:
        return (pair[1], pair[0])
    return pair

def get_measured_value_from_counts(counts_dict: dict) -> str:
    """ converts the dictionary counts measured to the
          value measured in string format
    """
    if len(list(counts_dict.keys())) != 1:
        raise ValueError('Circuit execution shots MUST be 1')
    return list(counts_dict.keys())[0]


def set_random_eta(eta_pair: Tuple[float, float]) -> int:
    """ return a random choice from attenuation pair with the correspondent index value """
    eta_value = random.choice(eta_pair)
    if eta_value == eta_pair[0]:
        return 0
    return 1


def check_value(real_index_eta: int, guess_index_eta: int):
    if real_index_eta == guess_index_eta:
        return 1
    return 0

In [34]:
def _play_and_guess_one_case(channel_configuration: dict) -> int:
    """ Execute a real execution with a random eta from the two passed,
        guess which one was used on the execution and
        check the result.
        Returns 1 on success (it was a correct guess) or 0 on fail (it was an incorrect guess)
    """
    eta_pair_index_to_use = set_random_eta(channel_configuration['eta_pair'])
    eta_pair_index_guessed = _compute_damping_channel(channel_configuration, eta_pair_index_to_use)
    return check_value(eta_pair_index_to_use, eta_pair_index_guessed)

In [35]:
def compute_average_success_probability(configuration: dict,
                                        plays: Optional[int] = 100) -> float:
        """ Computes the average success probability of running a specific configuration for the number of plays
            defined in the configuration.
        """

        reordered_configuration = { 'state_probability': configuration['state_probability'],
            'angle_rx': configuration['angle_rx'],
            'angle_ry': configuration['angle_ry'],
            'eta_pair': reorder_pair(configuration['eta_pair'])
        }

        success_counts = 0
        for play in range(plays):
            success_counts += _play_and_guess_one_case(reordered_configuration)

        return (success_counts / plays)

## tests

In [46]:
configuration = { 'state_probability': 1.0,
                 'angle_rx': 0,
                 'angle_ry': 0,
                 'eta_pair': (0, np.pi/2)}

In [48]:
compute_average_success_probability(configuration=configuration, plays=1000)

0.497